# Model Eğitimi

Bu notebook hazırlanan verileri kullanarak modeli eğitir.

**İçerik:**
1. Veri yükleme (NPY Dataset)
2. Model oluşturma
3. Eğitim döngüsü
4. Checkpoint kaydetme

---

## Önkoşullar

Çalıştırmadan önce:
- `02_data_preprocessing.ipynb` ile verileri hazırlayın
- GPU kullanımı önerilir (Colab Pro veya lokal GPU)

In [ ]:
# Google Colab için
from google.colab import drive
drive.mount('/content/drive')

## 1. Model Mimarisini Yükle

İlk notebook'tan model tanımlarını alıyoruz.

In [ ]:
%run 01_model_architecture.ipynb

## 2. Veri Yükleyici

Keras Sequence API kullanarak memory-efficient veri yükleme. Her epoch sonunda shuffle yapılıyor.

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# === AYARLAR ===
# Batch size'ı 4 yaptık, hem hızlı olur hem de RAM'i patlatmaz.
BATCH_SIZE = 4
IMG_WIDTH = 256
IMG_HEIGHT = 256

# === GÜNCELLENMİŞ SHUFFLE ÖZELLİKLİ DATASET SINIFI ===
class NPYDataset(tf.keras.utils.Sequence):
    def __init__(self, file_list_A, file_list_B, batch_size=1, shuffle=True):
        self.files_A = np.array(file_list_A) # Numpy array yapıyoruz
        self.files_B = np.array(file_list_B)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.files_A)) # İndeks listesi

        print(f"Veri listesi hazırlandı... ({len(self.files_A)} adet dosya)")

        # Başlarken bir kere karıştıralım
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.files_A) / self.batch_size))

    def __getitem__(self, index):
        # O anki batch için indeksleri seç
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        batch_A = []
        batch_B = []

        for k in indexes:
            # Dosyayı yükle
            imgA = np.load(self.files_A[k])
            imgB = np.load(self.files_B[k])

            # Boyut Kontrolü (H, W) -> (H, W, 1)
            if imgA.ndim == 2:
                imgA = np.expand_dims(imgA, axis=-1)
            if imgB.ndim == 2:
                imgB = np.expand_dims(imgB, axis=-1)

            batch_A.append(imgA)
            batch_B.append(imgB)

        return np.array(batch_A), np.array(batch_B)

    def on_epoch_end(self):
        # Her epoch sonunda verileri karıştır (Shuffle)
        if self.shuffle:
            np.random.shuffle(self.indexes)

## 3. Verileri Yükle

In [ ]:
# === VERİYİ OKUMA VE AYIRMA ===
dataset_path = r"/content/drive/MyDrive/Tasarım Dersi/Projects/data/processed_data_npy"

# Tüm dosya yollarını diskten alıyoruz
all_files_A = sorted(glob.glob(os.path.join(dataset_path, 'trainA') + '/*.npy'))
all_files_B = sorted(glob.glob(os.path.join(dataset_path, 'trainB') + '/*.npy'))

print(f"Toplam bulunan dosya (trainA): {len(all_files_A)}")
print(f"Toplam bulunan dosya (trainB): {len(all_files_B)}")

# Dosya isimlerini al
filenames_A = {os.path.basename(f) for f in all_files_A}
filenames_B = {os.path.basename(f) for f in all_files_B}

# Eksik dosyaları bul
missing_in_B = filenames_A - filenames_B
missing_in_A = filenames_B - filenames_A

if missing_in_A:
    print(f"HATA: trainA içinde karşılığı olmayan dosya(lar) (trainB'de var): {missing_in_A}")
if missing_in_B:
    print(f"HATA: trainB içinde karşılığı olmayan dosya(lar) (trainA'da var): {missing_in_B}")

# Sadece eşleşen dosyaları al
common_filenames = sorted(list(filenames_A.intersection(filenames_B)))

# Yeni dosya listelerini oluştur
filtered_files_A = [os.path.join(dataset_path, 'trainA', f) for f in common_filenames]
filtered_files_B = [os.path.join(dataset_path, 'trainB', f) for f in common_filenames]

print(f"Eşleşen dosya sayısı: {len(filtered_files_A)}")

# %90 Train, %10 Validation ayırma
train_A, val_A, train_B, val_B = train_test_split(filtered_files_A, filtered_files_B, test_size=0.10, random_state=42)

print(f"Eğitim Seti: {len(train_A)} adet")
print(f"Test/Doğrulama Seti: {len(val_A)} adet")

# === DATASETLERİ OLUŞTURMA (Shuffle Eklendi) ===
# Train setini karıştırıyoruz (True), Validation sabit kalabilir (False)
train_dataset = NPYDataset(train_A, train_B, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = NPYDataset(val_A, val_B, batch_size=BATCH_SIZE, shuffle=False)

## 4. GPU Kontrolü

In [ ]:
# === GPU KONTROL ===

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## 5. Eğitim Ayarları ve Callback

Eğitim sırasında:
- Her epoch sonunda örnek görseller kaydedilir
- Her 5 epoch'ta model checkpoint'u alınır

In [ ]:
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# === AYARLAR ===
LEARNING_RATE_G = 2e-4
LEARNING_RATE_D = 2e-4

# === 1. ADIM: KLASÖRLER ===
project_root = r"D:\Projects"
results_dir = os.path.join(project_root, "results")
checkpoint_dir = os.path.join(project_root, "model_checkpoints")

os.makedirs(results_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# === 2. ADIM: MODELİ DERLE ===
hybrid_gan = WGAN_GP_Pix2Pix(generator=generator, discriminator=discriminator)

hybrid_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_D, beta_1=0.5, beta_2=0.9),
    g_optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_G, beta_1=0.5, beta_2=0.9)
)

# === 3. ADIM: GÖRSEL İZLEME VE KAYIT ===
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, val_dataset, num_img=3):
        self.val_dataset = val_dataset
        self.num_img = num_img

    def on_epoch_end(self, epoch, logs=None):
        # --- A. GÖRSEL KAYIT (HER EPOCH) ---
        print(f"\nEpoch {epoch+1} bitti. Görseller işleniyor...")

        try:
            # Rastgele veri çek
            idx = np.random.randint(0, len(self.val_dataset))
            inp, tar = self.val_dataset[idx]
            prediction = self.model.generator(inp, training=False)

            title = ['Input (LD)', 'Generated (AI)', 'Target (HD)']
            display_count = min(self.num_img, inp.shape[0])

            for i in range(display_count):
                img_list = [inp[i], prediction[i], tar[i]]

                plt.figure(figsize=(12, 4))
                for j in range(3):
                    plt.subplot(1, 3, j+1)
                    plt.title(title[j])

                    # Veriyi alıp en küçük ve en büyüğe göre 0-1 arasına çekiyoruz.
                    # Bu sayede veri bozuk bile olsa ekranda gri gözükür.
                    img_data = img_list[j][:, :, 0]
                    _min, _max = np.min(img_data), np.max(img_data)

                    if _max - _min > 0:
                        show_img = (img_data - _min) / (_max - _min)
                    else:
                        show_img = img_data

                    plt.imshow(show_img, cmap='gray')
                    plt.axis('off')

                filename = f"epoch_{epoch+1}_{i}.png"
                save_path = os.path.join(results_dir, filename)
                plt.savefig(save_path)
                plt.close()

            # --- B. MODEL KAYIT (HER 5 EPOCH) ---
            if (epoch+1) % 5 == 0:
                model_name = f"G_epoch_{epoch+1}.h5"
                ckpt_path = os.path.join(checkpoint_dir, model_name)
                self.model.generator.save_weights(ckpt_path)
                print(f"✅ Model Kaydedildi: {ckpt_path}")

        except Exception as e:
            print(f"Kayıt hatası: {e}")

## 6. Eğitimi Başlat

50 epoch eğitim yaklaşık 4-6 saat sürer (GPU'ya bağlı).

In [ ]:
print("Eğitim başlıyor...")

hybrid_gan.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[GANMonitor(val_dataset)]
)